In [1]:
#importing modules
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#twitter api keys
auth = tweepy.OAuth1UserHandler(
   'ObDcu9DovwyYkx5w5tJouFGvP', '6A2UhUI9YpVis96xea1xmZqR5b2JIMIpUAOQIjJBaNPkINwnOj', '459226572-g5WB6p6QAcVciJ50uN6SLTRVfZKDqS0Bcp4bFGqd', 'jzFTq24nII5LUsG0FjsSCsrS0GLWN5ZK5BPIMfiJceUls')

api = tweepy.API(auth, wait_on_rate_limit=True)


In [ ]:
search_words = "#wakandaforever OR #WakandaForever OR #blackpanther OR #WakandaForeverID OR #BlackPantherWakandaForever OR #BlackPanther OR #BlackPanther2"
